# Sports14 Text/Image Feature Extraction

In [1]:

import os
import numpy as np
import pandas as pd

In [2]:
dataset = 'sports'
rslt_dir = f'/home/yueqi/peilin/MMRec/data/AmazonReview/{dataset}/'
image_dir = f'/home/yueqi/peilin/MMRec/AmazonRawData/Amazon_Review_Plus/photos/{dataset}/'
image_features = 'image_features_{dataset}.b'
os.chdir(rslt_dir)
os.getcwd()
i_id_mapping = 'i_id_mapping.csv'
meta_data_path = rslt_dir + f'meta-{dataset}.csv'

## Load text data

In [3]:
i_id, desc_str = 'itemID', 'description'


df = pd.read_csv(meta_data_path)
df.sort_values(by=[i_id], inplace=True)

print('data loaded!')
print(f'shape: {df.shape}')

df[:3]

data loaded!
shape: (18357, 10)


,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,1,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,21iMxsyDBRL._SX300_.jpg,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...
11982,2,B0048KGFHU,Tipton Polymer Gun Cleaning Picks,6.34,31K7ShuOAWL._SY300_.jpg,"{'also_bought': ['B006T6Y56E', 'B00162OKDY', '...",Tipton,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 501},Here's a clever idea that belongs in every cle...
15852,3,B0081JJVUC,TekMat 11-Inch X 17-Inch Handgun Cleaning Mat ...,10.67,51gB5DgFUBL._SY300_.jpg,"{'also_bought': ['B0014VX2M2', 'B0036N474S', '...",TekMat,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 9061},Your gun is a proud possession as well as an i...


In [4]:

# sentences: title + brand + category + description | All have title + description

title_na_df = df[df['title'].isnull()]
print(title_na_df.shape)

desc_na_df = df[df['description'].isnull()]
print(desc_na_df.shape)

na_df = df[df['description'].isnull() & df['title'].isnull()]
print(na_df.shape)

na3_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull()]
print(na3_df.shape)

na4_df = df[df['description'].isnull() & df['title'].isnull() & df['brand'].isnull() & df['categories'].isnull()]
print(na4_df.shape)

(91, 10)
(2659, 10)
(40, 10)
(40, 10)
(0, 10)


In [5]:

df[desc_str] = df[desc_str].fillna(" ")
df['title'] = df['title'].fillna(" ")
df['brand'] = df['brand'].fillna(" ")
df['categories'] = df['categories'].fillna(" ")


In [6]:
sentences = []
for i, row in df.iterrows():
    sen = row['title'] + ' ' + row['brand'] + ' '
    cates = eval(row['categories'])
    if isinstance(cates, list):
        for c in cates[0]:
            sen = sen + c + ' '
    sen += row[desc_str]
    sen = sen.replace('\n', ' ')

    sentences.append(sen)

sentences[:10]

['Ghost Inc Glock Armorers Tool 3/32 Punch Ghost S p o r t s   &   O u t d o o r s Ghost Armorer Tool (1). The GAT is made with a spring steel punch. The diameter is 3/32 of an inch or 2.5mm, this is the same as the OEM tool size. The difference is you will be able to press harder without bending the shaft of this punch. Just a better tool to work on your Glock with.',
 "Tipton Polymer Gun Cleaning Picks Tipton S p o r t s   &   O u t d o o r s Here's a clever idea that belongs in every cleaning kit - Gun-Cleaning Picks. This set of four picks with different configurations on each end provides eight different combinations to get into those tight, hard to reach places. Made of high-strength polymer so they won't scratch the finest surface. Set of four picks.",
 "TekMat 11-Inch X 17-Inch Handgun Cleaning Mat with Glock Imprint, Black TekMat S p o r t s   &   O u t d o o r s Your gun is a proud possession as well as an investment and should be protected as such. TekMats depict the fascina

In [7]:

course_list = df[i_id].tolist()
#sentences = df[desc_str].tolist()
lastID = course_list[-1]
lastIdx = len(course_list) - 1
print(f"lastID: {lastID} lastIdx: {lastIdx}")
assert course_list[-1] == len(course_list)

lastID: 18357 lastIdx: 18356


In [8]:
# should `pip install sentence_transformers` first
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')

sentence_embeddings = model.encode(sentences)
print('text encoded!')

assert sentence_embeddings.shape[0] == df.shape[0]
np.save(os.path.join(rslt_dir, 'text_feat.npy'), sentence_embeddings)
print('done!')


/home/yueqi/anaconda3/envs/MMRec/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


text encoded!
done!


In [9]:
sentence_embeddings[:10]

array([[-0.1321447 ,  0.03381675, -0.01577879, ..., -0.09672257,
         0.052081  ,  0.06492253],
       [-0.10782257, -0.02839002,  0.0622814 , ..., -0.04174746,
         0.01380114,  0.04938785],
       [-0.10046259,  0.12085465, -0.02257576, ..., -0.10255246,
         0.01178748, -0.00671982],
       ...,
       [-0.06964527, -0.04203182, -0.06922724, ..., -0.04277774,
         0.03630504,  0.05333261],
       [ 0.01461751,  0.04413512, -0.07554795, ..., -0.09187356,
        -0.13955116, -0.01189312],
       [-0.07896899, -0.00106236, -0.04540033, ..., -0.11145744,
        -0.03114022,  0.08727177]], dtype=float32)

In [10]:
load_txt_feat = np.load('text_feat.npy', allow_pickle=True)
print(load_txt_feat.shape)
load_txt_feat[:10]

(18357, 384)


array([[-0.1321447 ,  0.03381675, -0.01577879, ..., -0.09672257,
         0.052081  ,  0.06492253],
       [-0.10782257, -0.02839002,  0.0622814 , ..., -0.04174746,
         0.01380114,  0.04938785],
       [-0.10046259,  0.12085465, -0.02257576, ..., -0.10255246,
         0.01178748, -0.00671982],
       ...,
       [-0.06964527, -0.04203182, -0.06922724, ..., -0.04277774,
         0.03630504,  0.05333261],
       [ 0.01461751,  0.04413512, -0.07554795, ..., -0.09187356,
        -0.13955116, -0.01189312],
       [-0.07896899, -0.00106236, -0.04540033, ..., -0.11145744,
        -0.03114022,  0.08727177]], dtype=float32)

# Image encoder (V0)，following LATTICE, averaging over for missed items

In [11]:
df[:5]

,itemID,asin,title,price,imUrl,related,brand,categories,salesRank,description
0,1,1881509818,Ghost Inc Glock Armorers Tool 3/32 Punch,9.99,21iMxsyDBRL._SX300_.jpg,"{'also_bought': ['B000U3YWEM', 'B000U401J6', '...",Ghost,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 172909},Ghost Armorer Tool (1). The GAT is made with a...
11982,2,B0048KGFHU,Tipton Polymer Gun Cleaning Picks,6.34,31K7ShuOAWL._SY300_.jpg,"{'also_bought': ['B006T6Y56E', 'B00162OKDY', '...",Tipton,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 501},Here's a clever idea that belongs in every cle...
15852,3,B0081JJVUC,TekMat 11-Inch X 17-Inch Handgun Cleaning Mat ...,10.67,51gB5DgFUBL._SY300_.jpg,"{'also_bought': ['B0014VX2M2', 'B0036N474S', '...",TekMat,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 9061},Your gun is a proud possession as well as an i...
3320,4,B000N8OIE8,Lee Precision Powder Measure Kit,11.46,41T-weQ9X5L._SX300_.jpg,"{'also_bought': ['B000NOUEUO', 'B0013RD6OQ', '...",Lee,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 6842},Lee Powder Measure Dipper Kit includes 15 grad...
13372,5,B004Y27DVY,Glock Magazine Speed Loader for 9mm / .40 / .3...,7.88,41dpYrCCntL._SX300_.jpg,"{'also_bought': ['B0014VX2M2', 'B000U3YWEM', '...",Glock,"['Sports & Outdoors', 'Hunting & Fishing', 'Hu...",{'Sports &amp; Outdoors': 5930},Stop busting your fingers loading Glock Magazi...


In [12]:
import array

def readImageFeatures(path):
  f = open(path, 'rb')
  while True:
    asin = f.read(10).decode('UTF-8')
    if asin == '': break
    a = array.array('f')
    a.fromfile(f, 4096)
    yield asin, a.tolist()

In [13]:

img_data = readImageFeatures(image_features)
item2id = dict(zip(df['asin'], df['itemID']))

feats = {}
avg = []
for d in img_data:
    if d[0] in item2id:
        feats[int(item2id[d[0]])] = d[1]
        avg.append(d[1])
avg = np.array(avg).mean(0).tolist()

ret = []
non_no = []
for i in range(len(item2id)):
    if i in feats:
        ret.append(feats[i])
    else:
        non_no.append(i)
        ret.append(avg)

print('# of items not in processed image features:', len(non_no))
assert len(ret) == len(item2id)
np.save('image_feat.npy', np.array(ret))
np.savetxt("missed_img_itemIDs.csv", non_no, delimiter =",", fmt ='%d')
print('done!')

# of items not in processed image features: 181
done!
